In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

ERROR: (gcloud.dataproc.clusters.list) The required property [project] is not currently set.
It can be set on a per-command basis by re-running your command with the [--project] flag.

You may set it for your current workspace by running:

  $ gcloud config set project VALUE

or it can be set temporarily by the environment variable [CLOUDSDK_CORE_PROJECT]


In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
import builtins
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yonat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
!ls -l /usr/lib/spark/jars/graph*

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

NameError: name 'spark' is not defined

In [ ]:
bucket_name = '207044777' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

In [ ]:
parquetFile.count()

In [ ]:
%cd -q /home/dataproc
!ls inverted_index_gcp.py

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

In [ ]:
def tokenizer(text):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  tokens = [token for token in tokens if token not in all_stopwords]
  return tokens

def word_count(text, id):
  tokens = tokenizer(text)
  token_counts = Counter(tokens)
  return [(token,(id,count)) for token,count in token_counts.items()] 

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl,key=itemgetter(0))

def calculate_df(postings):
  return postings.map(lambda x: (x[0], len(x[1])))

def partition_postings_and_write(postings):
  return postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey().map(lambda x : InvertedIndex.write_a_posting_list(x,bucket_name))

In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start
# doc_word_counts = doc_text_pairs.map(lambda x: word_count(x[0], x[1]))
print(index_const_time)

In [ ]:
# create dictionary with doc_id as key and text length as value
doc_lengths = doc_text_pairs.map(lambda x: (x[1], len(tokenizer(x[0])))).collectAsMap()
print(len(doc_lengths))

In [ ]:
# make a dictionary with doc_id as key and normalized tf as value
# exp: doc id: 1, doc: 'i love apple apple', value: sqrt((1**2+1**2+2**2)/len(doc)**2))
import math

def normalize_tf(doc_id, doc):
    doc_tf = Counter(tokenizer(doc))
    doc_length = doc_lengths[doc_id]
    if doc_length != 0:
        return math.sqrt(builtins.sum([tf**2 for tf in doc_tf.values()])/doc_length**2)

# create a dictionary with doc_id as key and normalized tf as value
doc_norm_tf = doc_text_pairs.map(lambda x: (x[1], normalize_tf(x[1], x[0]))).collectAsMap()

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_body'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [ ]:
# download doc_tf to local and gcs
bucket_name = 'pkl_207044777'
tf = 'id2tf.pkl'
with open(tf, 'wb') as f:
    pickle.dump(doc_norm_tf, f)
# upload to gcs specific bucket
!gsutil cp {tf} gs://{bucket_name}/{tf}

In [ ]:
# Create inverted index instance
inverted = InvertedIndex('body_207044777')
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.DL = doc_lengths
# write the global stats out
inverted.write_index('.', 'body_index')
# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst